In [21]:
import math
from datetime import timedelta

import pandas as pd

In [22]:
# General restrictions:
start_date = "2025-01-01"
num_days = 365

employee_restrictions = {
    "hours_per_shift": 7,
    "max_hours_week_employee": 37.5,
    "max_hours_year_employee": 1852.5,  # TODO: Pending to check
    "min_weekend_rest_month_employee": 1,
    "max_timeoff_employee": 2,
    "shifts": ["M", "T"],
    "max_persons_per_shift": {
        "M": 1,
        "T": 1,
    },
}

In [23]:
# Employees information
employees_full = [
    {
        "capacity": 1,
    }
]

employees_partial = [
    {
        "capacity": 0.77,
    }
]

employees_temp = employees_full * 2 + employees_partial * 1

employees = []
for index, one_employee in enumerate(employees_temp):
    employees.append(
        {
            "name": f"E{index + 1}",
            "capacity": one_employee["capacity"],
            "max_hours_year": math.ceil(
                employee_restrictions["max_hours_year_employee"]
                * one_employee["capacity"]
            ),
            "max_hours_week": math.ceil(
                employee_restrictions["max_hours_week_employee"]
                * one_employee["capacity"]
            ),
        }
    )

employees

[{'name': 'E1', 'capacity': 1, 'max_hours_year': 1853, 'max_hours_week': 38},
 {'name': 'E2', 'capacity': 1, 'max_hours_year': 1853, 'max_hours_week': 38},
 {'name': 'E3',
  'capacity': 0.77,
  'max_hours_year': 1427,
  'max_hours_week': 29}]

In [24]:
# Init employees with dates:
dates = pd.date_range(start=start_date, periods=num_days, freq="D")
employees_info = pd.DataFrame(
    index=dates, columns=[emp["name"] for emp in employees], data=""
)
employees_info

,E1,E2,E3
2025-01-01,,,
2025-01-02,,,
2025-01-03,,,
2025-01-04,,,
2025-01-05,,,
...,...,...,...
2025-12-27,,,
2025-12-28,,,
2025-12-29,,,
2025-12-30,,,


In [25]:
# Init all employees by shift
all_employees_by_shift = pd.DataFrame(
    index=dates, columns=[one_shift for one_shift in employee_restrictions["shifts"]]
)
all_employees_by_shift[:] = 0
all_employees_by_shift

,M,T
2025-01-01,0,0
2025-01-02,0,0
2025-01-03,0,0
2025-01-04,0,0
2025-01-05,0,0
...,...,...
2025-12-27,0,0
2025-12-28,0,0
2025-12-29,0,0
2025-12-30,0,0


In [26]:
# Fill information:

for date in all_employees_by_shift.index:
    for shift in all_employees_by_shift.columns:
        if (
            all_employees_by_shift.loc[date, shift]
            >= employee_restrictions["max_persons_per_shift"][shift]
        ):  # No more employees needed
            continue

        for employee in employees_info.columns:
            if not employees_info.loc[date, employee]:
                seven_days_ago = date - timedelta(days=7)
                last_7_days_employee = employees_info.loc[seven_days_ago:date, employee]
                shift_counts = last_7_days_employee.value_counts().reindex(
                    ["M", "T"], fill_value=0
                )
                total_worked_days_in_7_days = shift_counts.sum()
                employee_capacity = next(
                    emp["capacity"] for emp in employees if emp["name"] == employee
                )
                if (
                    (total_worked_days_in_7_days + 1) * employee_restrictions["hours_per_shift"]
                    >= employee_restrictions["max_hours_week_employee"]
                    * employee_capacity
                ):
                    continue
                all_employees_by_shift.loc[date, shift] += 1
                employees_info.loc[date, employee] = shift
                if (
                    all_employees_by_shift.loc[date, shift]
                    >= employee_restrictions["max_persons_per_shift"][shift]
                ):  # No more employees needed
                    break


In [27]:
all_employees_by_shift.index = pd.to_datetime(all_employees_by_shift.index)
all_employees_by_shift.index = all_employees_by_shift.index.strftime("%Y-%m-%d")
all_employees_by_shift

,M,T
2025-01-01,1,1
2025-01-02,1,1
2025-01-03,1,1
2025-01-04,1,1
2025-01-05,1,1
...,...,...
2025-12-27,1,1
2025-12-28,1,1
2025-12-29,1,1
2025-12-30,1,1


In [28]:
employees_info.index = pd.to_datetime(employees_info.index)
employees_info.index = employees_info.index.strftime("%Y-%m-%d")
employees_info

,E1,E2,E3
2025-01-01,M,T,
2025-01-02,M,T,
2025-01-03,M,T,
2025-01-04,M,T,
2025-01-05,M,T,
...,...,...,...
2025-12-27,M,T,
2025-12-28,M,T,
2025-12-29,M,T,
2025-12-30,M,T,


In [29]:
employees_info.index = pd.to_datetime(employees_info.index)
employees_info.index = employees_info.index.strftime("%Y-%m-%d")
employees_info[["E1", "E2", "E3"]]

,E1,E2,E3
2025-01-01,M,T,
2025-01-02,M,T,
2025-01-03,M,T,
2025-01-04,M,T,
2025-01-05,M,T,
...,...,...,...
2025-12-27,M,T,
2025-12-28,M,T,
2025-12-29,M,T,
2025-12-30,M,T,


In [30]:
output_filename = "../samples/m_a_2025.xlsx"
employees_info.index = pd.to_datetime(employees_info.index)
employees_info.index = employees_info.index.strftime("%Y-%m-%d")
employees_info.to_excel(output_filename, sheet_name="Shift Schedule")

In [31]:
# Suponiendo que employees_info ya está definido y tiene un índice de fechas
employees_info.index = pd.to_datetime(employees_info.index)

# Extraer el mes, el día del mes y el día de la semana en español
month = employees_info.index.month
day_of_month = employees_info.index.day
days_of_week_map = {0: "L", 1: "M", 2: "X", 3: "J", 4: "V", 5: "S", 6: "D"}
months_map = {
    1: "Enero",
    2: "Febrero",
    3: "Marzo",
    4: "Abril",
    5: "Mayo",
    6: "Junio",
    7: "Julio",
    8: "Agosto",
    9: "Septiembre",
    10: "Octubre",
    11: "Noviembre",
    12: "Diciembre",
}
month = employees_info.index.month.map(months_map)
day_of_week = employees_info.index.dayofweek.map(days_of_week_map)

# Crear MultiIndex para las filas
multi_index_index = pd.MultiIndex.from_arrays(
    [month, day_of_week, day_of_month], names=["", "", ""]
)

# Asignar el MultiIndex al índice del DataFrame
employees_info.index = multi_index_index

# Transponer el DataFrame
transposed_employees_info = employees_info.T
transposed_employees_info

Enero                             ... Diciembre                           
       X  J  V  S  D  L  M  X  J  V  ...         L  M  X  J  V  S  D  L  M  X
      1  2  3  4  5  6  7  8  9  10  ...        22 23 24 25 26 27 28 29 30 31
E1     M  M  M  M  M           M  M  ...         M  M           M  M  M  M  M
E2     T  T  T  T  T           T  T  ...         T  T           T  T  T  T  T
E3                    M  M  M        ...               M  M  M               

[3 rows x 365 columns]

In [32]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import (
    Border,
    Font,
    PatternFill,
    Side,
)

# Suponiendo que transposed_employees_info ya está definido
output_filename = "../samples/m_a_2025_transponse.xlsx"
transposed_employees_info.to_excel(output_filename, sheet_name="Shift Schedule")

# Abrir el archivo Excel con openpyxl
workbook = load_workbook(output_filename)
worksheet = workbook["Shift Schedule"]

# Eliminar la cuarta fila
worksheet.delete_rows(4)

# Ajustar la anchura de todas las columnas a un valor mínimo
min_width = 3  # Puedes ajustar este valor según tus necesidades
for col in worksheet.iter_cols():
    for cell in col:
        if not any(
            cell.coordinate in merged_cell
            for merged_cell in worksheet.merged_cells.ranges
        ):  # Filtrar celdas combinadas
            column = cell.column_letter  # Obtener la letra de la columna
            worksheet.column_dimensions[column].width = min_width
            break  # Ajustar solo una vez por columna

# Definir el estilo de relleno para la fila de meses
fill = PatternFill(start_color="0099FF", end_color="0099FF", fill_type="solid")
font = Font(color="FFFFFF", bold=True)

# Aplicar el estilo de relleno a la fila de meses
for cell in worksheet[1]:
    cell.fill = fill
    cell.font = font

# Definir el estilo de relleno amarillo para los fines de semana
weekend_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

# Definir el estilo de borde
thin_border = Border(
    left=Side(style="thin"),
    right=Side(style="thin"),
    top=Side(style="thin"),
    bottom=Side(style="thin"),
)

# Aplicar el estilo de relleno amarillo y los bordes a las columnas de "S" y "D"
for col in worksheet.iter_cols(
    min_row=2, max_row=worksheet.max_row, min_col=2, max_col=worksheet.max_column
):
    day_of_week_cell = col[
        0
    ]  # La primera celda de la columna (fila 2) contiene el día de la semana
    if day_of_week_cell.value in ["S", "D"]:
        for cell in col:
            cell.fill = weekend_fill

# Aplicar el borde a todas las celdas
for row in worksheet.iter_rows(
    min_row=1, max_row=worksheet.max_row, min_col=1, max_col=worksheet.max_column
):
    for cell in row:
        cell.border = thin_border


# Guardar los cambios en el archivo Excel
workbook.save(output_filename)